In [1]:
import os,sys #user to create and modify file name and save it
import pandas as pd #for dataframe activities
from datetime import datetime,timedelta #for the time related activities
import matplotlib.pyplot as plt #for the Ploting purpose
import numpy as np #numerical fucntions
import datetime 
import warnings #to ignore unnecessary warnings
import xarray as xr
import numpy as np
import regionmask
import geopandas as gp #ReadingShapefile
import glob                                                                 
import xlsxwriter #for writing in Excel
warnings.filterwarnings('ignore')

In [2]:
from scipy.interpolate import griddata
min_lon = 78.00
max_lon = 96.00
min_lat = 8.00
max_lat = 23.00

xi=np.arange(min_lon,max_lon+1,1)
yi=np.arange(min_lat,max_lat+1,1)

In [3]:
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')

#Clipping to shape File and Creating Mask File
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])


# mask=countries_mask_poly.mask(ds,lat_name='latitude',lon_name='longitude')

### For ERA

In [4]:
#For MeanWind and WindDirection
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/ERA5')
file='U-Wind/PreMonDaily_Uwind850.nc'
ds=xr.open_dataset(file)
ds=ds.reindex(latitude=list(reversed(ds.latitude)))
u=ds.u

file='V-Wind/PreMonDaily_Vwind850.nc'
ds=ds.reindex(latitude=list(reversed(ds.latitude)))
ds=xr.open_dataset(file)
v=ds.v

# Compute the wind speed and direction
MW = np.sqrt(u**2 + v**2)
WD = np.degrees(np.arctan2(u, v))
WD = (WD + 360) % 360

var=MW.interp(latitude=yi).interp(longitude=xi)
# var=MW.reindex(latitude=yi, longitude=xi, method='nearest')
MW_values=var.values.flatten()

var=WD.interp(latitude=yi).interp(longitude=xi)
# var=WD.reindex(latitude=yi, longitude=xi, method='nearest')

WD_values=var.values.flatten()



In [5]:
#For VerticalWind
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/ERA5/VerticalVelocity')
ds=xr.open_dataset('PreMonDaily_VerticalVelocity.nc')
ds=ds.reindex(latitude=list(reversed(ds.latitude)))
W=ds.w

var=W.interp(latitude=yi).interp(longitude=xi)
# var=W.reindex(latitude=yi, longitude=xi, method='nearest')
W_values=var.values.flatten()

In [6]:
#For LTS
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/ERA5/LTS')
ds=xr.open_dataset('PreMonDaily_LTS.nc')
LTS=ds.t

var=LTS.interp(latitude=yi).interp(longitude=xi)
# var=LTS.reindex(latitude=yi, longitude=xi, method='nearest')

LTS_values=var.values.flatten()

In [7]:
#For PBLH
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/PBLH')
ds=xr.open_dataset('PreMonDaily_PBLH.nc')
PBLH=ds.TCZPBL

var=PBLH.interp(lat=yi).interp(lon=xi)
# var=PBLH.reindex(lat=yi, lon=xi, method='nearest')
PBLH_values=var.values.flatten()

In [8]:
#For RH
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/ERA-5 RH/PreMon')
ds=xr.open_dataset('PreMonDaily_RHavg.nc')
# ds=ds.reindex(latitude=list(reversed(ds.latitude)))
RH=ds.r

var=RH.interp(latitude=yi).interp(longitude=xi)
# var=RH.reindex(latitude=yi, longitude=xi, method='nearest')

RH_values=var.values.flatten()

In [9]:
#Nd
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/Nd')
ds=xr.open_dataset('Nd_PreMon.nc')
ds=ds.interp(latitude=yi).interp(longitude=xi)
Nd=ds.__xarray_dataarray_variable__

Nd_values=Nd.values.flatten()

### For Merra

In [10]:
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/MERRA2')
file='Composites/N_MERRA-PreMon.nc'
ds=xr.open_dataset(file)
ds=ds.interp(lat=yi).interp(lon=xi)
# ds=ds.reindex(lat=yi, lon=xi, method='nearest')


sm=ds.BC_AOD+ds.OC_AOD+ds.SULFATE_AOD
dst=ds.SEASALT_AOD+ds.DUST_AOD


sm_values=sm.values.flatten()
dst_values=dst.values.flatten()


# file='MERRA-PreMon.nc'
# ds=xr.open_dataset(file)

# AOD=ds.AOD
# AOD_values=[]
# for lat, lon in lat_lon_pairs:
#     AOD1=AOD.sel(lon=lon, lat=lat, method='nearest')
#     AOD_values=np.append(AOD_values, AOD1.values.flatten())



In [11]:
# For AI
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/MERRA2')
file='MERRA-PreMon.nc'
ds=xr.open_dataset(file)
ds=ds.interp(lat=yi).interp(lon=xi)
# ds=ds.reindex(lat=yi, lon=xi, method='nearest')


AI=ds.AI
AI_values=AI.values.flatten()

### For MODIS

In [12]:
# For Cloud Drop Effective Radius
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/MODIS')
file='FINAL_MODIS_PreMon.nc'
ds=xr.open_dataset(file)
ds=ds.interp(latitude=yi).interp(longitude=xi)
# ds=ds.reindex(latitude=yi, longitude=xi, method='nearest')


CR=ds.Cloud_Effective_Radius_Liquid_Mean
CR_values=CR.values.flatten()

CTP=ds.Cloud_Top_Pressure_Mean
CTP_values=CTP.values.flatten()

COT=ds.Cloud_Optical_Thickness_Combined_Mean
COT_values=COT.values.flatten()

CTT=ds.Cloud_Top_Temperature_Mean
CTT_values=CTT.values.flatten()


# AOD=ds.AOD_550_Dark_Target_Deep_Blue_Combined_Mean
# AOD_values=[]
# for lat, lon in lat_lon_pairs:
#     AOD1=AOD.sel(longitude=lon, latitude=lat, method='nearest')
#     AOD_values=np.append(AOD_values, AOD1.values.flatten())

In [13]:
# Liquid Water Path
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/LWP')
file='LWP_MODIS_PreMon.nc'
ds=xr.open_dataset(file)
ds=ds.interp(latitude=yi).interp(longitude=xi)
# ds=ds.reindex(latitude=yi, longitude=xi, method='nearest')

LWP=ds.LWP
LWP_values=LWP.values.flatten()

In [14]:
# Precipitable water
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/PrecipitableWater-MODIS')
file='PreMonDaily_PrecipWater.nc'
ds=xr.open_dataset(file)
ds=ds.interp(latitude=yi).interp(longitude=xi)
# ds=ds.reindex(latitude=yi, longitude=xi, method='nearest')


PW=ds.Atmospheric_Water_Vapor_QA_Mean
PW_values=PW.values.flatten()


## Write it to Excel

### ML3

In [15]:
import xarray as xr
import numpy as np
import xlsxwriter
os.chdir('/Volumes/PtatoBasket/ACIML/Outputs/Sheets_of_ML/')


In [16]:
# import csv 

# # name of csv file 
# filename = "ML4-PreMon.csv"
# fields=['PBLH' ,
#                     'WindDirection-850', 'MeanWind-850', 'RH', 'V-850', 'SMOKE-AOD',
#                     'DUST-AOD', 'AI','PrecipitableVapour', 'PreciCal', 'LTS', 'Nd', 'CloudRadius', 'LWP']
# # writing to csv file 
# with open(filename, 'w') as csvfile: 
#     # creating a csv dict writer object 
#     writer = csv.DictWriter(csvfile, fieldnames = fields) 
        
#     # writing headers (field names) 
#     writer.writeheader() 
        
#     # writing data rows 
#     writer.writerows(mydict) 


In [17]:
# %%script echo skipping
# Create Excel workbook and worksheet
workbook = xlsxwriter.Workbook('ML5.1-PreMonM3.xlsx')
worksheet = workbook.add_worksheet(name='Sheet1')


# Write column headers
worksheet.write_row(0, 0, ['PBLH' ,
                    'WindDirection-850', 'MeanWind-850', 'RH', 'V-850', 'SMOKE-AOD',
                    'DUST-AOD', 'AI','PrecipitableVapour', 'LTS', 'Nd', 'CloudRadius', 'LWP','CTP','CTT','COT'])


i=0
for x in range(len(LWP_values)):
    if not any([np.isnan(LWP_values[x]),
           np.isnan(CR_values[x]),
           np.isnan(Nd_values[x]),
           np.isnan(LTS_values[x]),
           np.isnan(PW_values[x]),
           np.isnan(AI_values[x]),
           np.isnan(dst_values[x]),
           np.isnan(sm_values[x]),
           np.isnan(W_values[x]),
           np.isnan(RH_values[x]),
           np.isnan(MW_values[x]),
           np.isnan(WD_values[x]),
           np.isnan(PBLH_values[x]),
           np.isnan(CTP_values[x]),
           np.isnan(CTT_values[x]),
           np.isnan(COT_values[x]),

           ]):
            i=i+1
            worksheet.write_number(i,0,PBLH_values[x])
            worksheet.write_number(i,1,WD_values[x])
            worksheet.write_number(i,2,MW_values[x])
            worksheet.write_number(i,3,RH_values[x])
            worksheet.write_number(i,4,W_values[x])
            worksheet.write_number(i,5,sm_values[x])
            worksheet.write_number(i,6,dst_values[x])
            worksheet.write_number(i,7,AI_values[x])
            worksheet.write_number(i,8,PW_values[x])
            worksheet.write_number(i,9,LTS_values[x])
            worksheet.write_number(i,10,Nd_values[x])
            worksheet.write_number(i,11,CR_values[x])
            worksheet.write_number(i,12,LWP_values[x])
            worksheet.write_number(i,13,CTP_values[x])
            worksheet.write_number(i,14,CTT_values[x])
            worksheet.write_number(i,15,COT_values[x])


workbook.close()

In [18]:
%%script echo skipping

labels=['PBLH' ,'WindDirection-850', 'MeanWind-850', 'RH', 'VerticalVelocity-850', 'SMOKE-AOD','DUST-AOD', 'AI','PrecipitableVapour', 'LTS', 'Nd', 'CloudRadius','CTP','CTT','COT','LWP']
df=pd.DataFrame(data=[PBLH_values,WD_values,MW_values,RH_values,W_values,sm_values,dst_values,AI_values,PW_values,LTS_values,Nd_values,CR_values,CTP_values,CTT_values,COT_values,LWP_values]).T
df.columns=labels
df=df.dropna(how='any',axis=0).reset_index(drop=True)
df.to_excel('ML5.1-PreMonM2.xlsx')

skipping


# Stat Sheet

In [19]:
%%script echo skipping÷

os.chdir('/Volumes/PtatoBasket/ACIML/Outputs/Sheets_of_Stat')

# Create Excel workbook and worksheet
workbook = xlsxwriter.Workbook('STAT1-PreMonM2.xlsx')
worksheet = workbook.add_worksheet(name='Sheet1')


# Write column headers
worksheet.write_row(0, 0, ['AnthroPogenic-AOD',
                    'DUST-AOD', 'AI', 'Nd', 'CloudRadius','LWP','AOD'])

i=0
for x in range(len(LWP_values)):
    if not any([np.isnan(LWP_values[x]),
           np.isnan(CR_values[x]),
           np.isnan(Nd_values[x]),
           np.isnan(LTS_values[x]),
           np.isnan(PW_values[x]),
           np.isnan(AI_values[x]),
           np.isnan(dst_values[x]),
           np.isnan(sm_values[x]),
           np.isnan(W_values[x]),
           np.isnan(RH_values[x]),
           np.isnan(MW_values[x]),
           np.isnan(WD_values[x]),
           np.isnan(PBLH_values[x]),
           np.isnan(AOD_values[x])
           ]):    
            i=i+1
            # worksheet.write_number(i,0,PBLH_values[x])
            # worksheet.write_number(i,1,WD_values[x])
            # worksheet.write_number(i,2,MW_values[x])
            # worksheet.write_number(i,3,RH_values[x])
            # worksheet.write_number(i,4,W_values[x])
            worksheet.write_number(i,0,sm_values[x])
            worksheet.write_number(i,1,dst_values[x])
            worksheet.write_number(i,2,AI_values[x])
            # worksheet.write_number(i,8,PW_values[x])
            # worksheet.write_number(i,9,PC_values[x])
            # worksheet.write_number(i,10,LTS_values[x])
            worksheet.write_number(i,3,Nd_values[x])
            worksheet.write_number(i,4,CR_values[x])
            worksheet.write_number(i,5,LWP_values[x])
            worksheet.write_number(i,6,AOD_values[x])

workbook.close()

skipping÷
